In [6]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as p3
import numpy as np
import random
import time

from functools import partial
from ipywidgets import interact, RadioButtons, IntSlider, FloatSlider, Dropdown, BoundedFloatText
from numpy.linalg import norm

random.seed(42) # начальное состояние генератора случайных чисел, чтобы можно было воспроизводить результаты.

#Активационная функция сигмоидального нейрона
def sigmoid(x):
    return 1/(1+np.exp(-x))
    
#Производная активационной функции сигмоидального нейрона
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

#Считает целевую функциюю
def J_quadratic(neuron, X, y):
    """
    Оценивает значение квадратичной целевой функции.
    Всё как в лекции, никаких хитростей.

    neuron - нейрон, у которого есть метод vectorized_forward_pass, предсказывающий значения на выборке X
    X - матрица входных активаций (n, m)
    y - вектор правильных ответов (n, 1)

    Возвращает значение J (число)
    """

    assert y.shape[1] == 1, 'Incorrect y shape'

    return 0.5 * np.mean((neuron.vectorized_forward_pass(X) - y) ** 2)
    
#Вычисляет вектор частных производных целевой функции по каждому из предсказаний(используется при расчете градиента)
def J_quadratic_derivative(y, y_hat):
    """
    Вычисляет вектор частных производных целевой функции по каждому из предсказаний.
    y_hat - вертикальный вектор предсказаний,
    y - вертикальный вектор правильных ответов,
    
    В данном случае функция смехотворно простая, но если мы захотим поэкспериментировать 
    с целевыми функциями - полезно вынести эти вычисления в отдельный этап.
    
    Возвращает вектор значений производной целевой функции для каждого примера отдельно.
    """
    
    assert y_hat.shape == y.shape and y_hat.shape[1] == 1, 'Incorrect shapes'
    
    return (y_hat - y) / len(y)
    
    
#Рассчитывает градиент(аналитическая производная целевой функции)
def compute_grad_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
    """
    Аналитическая производная целевой функции
    neuron - объект класса Neuron
    X - вертикальная матрица входов формы (n, m), на которой считается сумма квадратов отклонений
    y - правильные ответы для примеров из матрицы X
    J_prime - функция, считающая производные целевой функции по ответам
    
    Возвращает вектор размера (m, 1)
    """
    
    # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    
    
    # осознайте эту строчку:
    dz_dw = X

    

    # а главное, эту:
    grad = ((dy_dyhat * dyhat_dz).T).dot(dz_dw)
    
    # можно было написать в два этапа. Осознайте, почему получается одно и то же
    #2)grad_matrix = dy_dyhat * dyhat_dz * dz_dw
    #2)grad = np.sum(grad_matrix, axis=0)
    
    # Сделаем из горизонтального вектора вертикальный
    
    
    grad = grad.T
    #2)grad.shape = (5,1)
    
    return grad

class Neuron:

    def __init__(self, weights, activation_function=sigmoid, activation_function_derivative = sigmoid_prime):
        
        assert weights.shape[1] == 1, "Incorrect weight shape"
        
        self.w = weights
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
        

    #Рассчет предсказанных значений без векторизации(хуже способ)
    def forward_pass(self, single_input):
        
        result = 0
        for i in range(self.w.size):
            result += float(self.w[i] * single_input[i])
        return self.activation_function(result)
        
    #Сумматорный метод
    def summatory(self, input_matrix):
        return input_matrix.dot(self.w)
    
    
    #Активационный метод
    def activation(self, summatory_activation):
        return self.activation_function(summatory_activation)
    
    
    #Векторизованный метод рассчета предсказанных значений(гораздо лучший способ)
    def vectorized_forward_pass(self, input_matrix):
        return self.activation(self.summatory(input_matrix))

    
    #Обучение весов с помощью одного батча(несколько случайных примеров из всех имеющихся примеров)
    def update_mini_batch(self, X, y, learning_rate, eps):
        
        before = J_quadratic(self, X, y)
        
        #self.w -= learning_rate * compute_grad_analytically(self,X,y)
        self.w -= learning_rate * compute_grad_numerically_2(self,X,y)

        
        return np.abs(before-J_quadratic(self, X, y)) < eps
    
    
    #Обучение весов с помощью батчей
    def SGD(self, X, y, batch_size, learning_rate =0.1, eps=1e-6, max_steps = 200):
        cur_step = 0
        #for _ in range(max_steps):
        while cur_step < max_steps:
            cur_step += 1
            batch = np.random.choice(len(X), batch_size, replace = False)
            if self.update_mini_batch(X[batch], y[batch], learning_rate, eps) == 1: 
                return 1
        return 0
    

#X = np.random.randint(-20,20,size = 15).reshape(5,3)
#X = np.arange(1,16).reshape(5,3)
#X[:,0] = 1

W = np.ones((3,1), dtype = int)
W = np.random.randint(-20,20,size = 3).reshape(3,1)
    
np.random.seed(42)
n = 10
m = 5

X = 20 * np.random.sample((n,m)) - 10
y = (np.random.random(n) < 0.5).astype(np.int)[:,np.newaxis]
w = 2 * np.random.random((m,1)) - 1


N = Neuron(w)
print("Веса: \n",N.w)

Веса: 
 [[-0.22264542]
 [-0.45730194]
 [ 0.65747502]
 [-0.28649335]
 [-0.43813098]]
